importing modules


In [ ]:

# Install libraries

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

print("Libraries imported successfully")
print("it is running")

c:\Users\DELL\Desktop\youtube_chatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully
it is running


indexing

In [3]:
#Step 1a - Indexing (Document Ingestion)
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

video_id = "QYAnJ_QyCQg"

try:
    fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    transcript_list = fetched_transcript.to_raw_data()
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    
    
except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"An error occurred: {e}")

In [4]:
#Step 1b - Indexing (Text Splitting)
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [5]:
#Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)
print("Vector store created successfully")
vector_store.index_to_docstore_id

C:\Users\DELL\AppData\Local\Temp\ipykernel_24376\2881360337.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 449.86it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector store created successfully


{0: '822705cc-b30c-48b6-afec-c6b0664e4bd1',
 1: 'e904d6eb-d861-4e95-92a5-dd4d98ab301c',
 2: 'b91fcf99-9188-4b2f-bc2e-f689f36422b2',
 3: '23ff2445-d44a-48a7-a14b-92eca6589796',
 4: '141d41ab-95cc-41db-97c9-2c1ef1b44432',
 5: 'a57579d9-cb0d-4bca-868c-38ec560194c8',
 6: 'da709ecb-c4f7-4e76-983e-e6a77cd7f765',
 7: '4114cfba-caaa-4654-8848-29b4bed342f3',
 8: '3cbad02c-5c90-4d5c-a18b-36148e7a16dd',
 9: 'aedc2667-203e-4d5d-96ff-baf29dd7ddee',
 10: '1e65b88a-f704-4568-8c51-e4ccf80d8bb0',
 11: '79a27f1a-db3f-4b1b-bd2b-c79951761d79',
 12: '2cd061de-b7c9-44c1-934f-a89318ce32d0',
 13: '0b115cc0-1bf6-4a19-94cd-42d025d22737',
 14: 'c7688073-ca39-41e6-8467-200dc519cc10'}

Retrieval

In [10]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000268E1A4C980> search_kwargs={'k': 4}


In [12]:
retriever.invoke('who is steve jobs?')

[Document(id='4114cfba-caaa-4654-8848-29b4bed342f3', metadata={}, page_content="to start over. I didn't see it then, but it turned out that getting fired from Apple was the best thing that could have ever happened to me. The heaviness of being successful was replaced by the lightness of being a beginner again, less sure about everything. It freed me to enter one of the most creative periods of my life. During the next 5 years, I started a company named Next, another company named Pixar, and fell in love with an amazing woman who would become my wife. Pixar went on to create the world's first computer animated feature film, Toy Story, and is now the most successful animation studio in the world. In a remarkable turn of events, Apple bought Next, and I returned to Apple. And the technology we developed at Next is at the heart of Apple's current renaissance. And Loren and I have a wonderful family together. I'm pretty sure none of this would have happened if I hadn't been fired from Apple

augmentation


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# This looks for the .env file and loads it into your system
load_dotenv()
# Now initialize the model without passing any extra arguments
llm = ChatOpenAI(model="gpt-4o")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [15]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [45]:
question          = "What is styeve jobs known for?"
retrieved_docs    = retriever.invoke(question)

In [46]:
retrieved_docs

[Document(id='0b115cc0-1bf6-4a19-94cd-42d025d22737', metadata={}, page_content='inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary. When I was young, there was an amazing publication called the Whole Earth Catalog, which was one of the Bibles of my generation. It was created by a fellow named Stuart Brand not far from here in Menllo Park. And he brought it to life with his poetic touch. This was in the late60s before personal computers and desktop publishing. So, it was all made with typewriters, scissors, and Polaroid cameras. It was sort of like Google in paperback form 35 years before Google came along. It was idealistic, overflowing with neat tools and great notions. Stuart and his team put out several issues of the Whole Earth catalog. And then when it had run its course, they put out a final issue. It was the mid 1970s and I was your age. On the back cover of t

In [47]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary. When I was young, there was an amazing publication called the Whole Earth Catalog, which was one of the Bibles of my generation. It was created by a fellow named Stuart Brand not far from here in Menllo Park. And he brought it to life with his poetic touch. This was in the late60s before personal computers and desktop publishing. So, it was all made with typewriters, scissors, and Polaroid cameras. It was sort of like Google in paperback form 35 years before Google came along. It was idealistic, overflowing with neat tools and great notions. Stuart and his team put out several issues of the Whole Earth catalog. And then when it had run its course, they put out a final issue. It was the mid 1970s and I was your age. On the back cover of their final issue was a photograph of an early morning country road, the kind\n\

In [48]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary. When I was young, there was an amazing publication called the Whole Earth Catalog, which was one of the Bibles of my generation. It was created by a fellow named Stuart Brand not far from here in Menllo Park. And he brought it to life with his poetic touch. This was in the late60s before personal computers and desktop publishing. So, it was all made with typewriters, scissors, and Polaroid cameras. It was sort of like Google in paperback form 35 years before Google came along. It was idealistic, overflowing with neat tools and great notions. Stuart and his team put out several issues of the Whole Earth catalog. And then when it 

generation


In [49]:
answer = llm.invoke(final_prompt)
print(answer.content)

Steve Jobs is known for co-founding Apple and playing a pivotal role in the development of the Macintosh computer, which was the first computer with beautiful typography. He also started companies like Next and Pixar, with Pixar creating the world's first computer-animated feature film, Toy Story. Additionally, he returned to Apple, where the technology developed at Next contributed to Apple's renaissance.


building chain


In [50]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [51]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [52]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [53]:
parallel_chain.invoke('who is steve jobs?')

{'context': 'to start over. I didn\'t see it then, but it turned out that getting fired from Apple was the best thing that could have ever happened to me. The heaviness of being successful was replaced by the lightness of being a beginner again, less sure about everything. It freed me to enter one of the most creative periods of my life. During the next 5 years, I started a company named Next, another company named Pixar, and fell in love with an amazing woman who would become my wife. Pixar went on to create the world\'s first computer animated feature film, Toy Story, and is now the most successful animation studio in the world. In a remarkable turn of events, Apple bought Next, and I returned to Apple. And the technology we developed at Next is at the heart of Apple\'s current renaissance. And Loren and I have a wonderful family together. I\'m pretty sure none of this would have happened if I hadn\'t been fired from Apple. It was awful tasting medicine, but I guess the patient neede

In [54]:
parser = StrOutputParser()

In [55]:
main_chain = parallel_chain | prompt | llm | parser

In [56]:
main_chain.invoke('Can you summarize the video')

'The transcript appears to be from a speech, likely given by Steve Jobs, where he shares personal anecdotes and life lessons. He talks about a publication that ended with the message "Stay hungry, stay foolish," which he wishes for the audience as they graduate. He recounts his experience with a cancer diagnosis, initially thought to be terminal, but later found to be a rare, curable form. This experience reinforced his belief in living life fully and following one\'s heart. He also reflects on being fired from Apple, which he describes as a blessing in disguise that led to a period of creativity and success, including founding Next and Pixar, and eventually returning to Apple. The overarching themes are about embracing life\'s uncertainties, learning from setbacks, and pursuing one\'s passions.'